## Cluster Analysis with Multinomial EM (Text Clustering)

#### In the K-means clustering algorithm we forced documents to be assigned to one cluster. This is known as hard clustering. This approach in not really accurate when working with language.  When someone writes about a topic they use a certain terminology to describe that topic. We assume that when someone else writes about the same topic they are likely to use the same terminology or words. But words may also be used across multiple topics and a clinical note may describe multiple topics. It may be more appropriate to assign words and documents to multiple topics (or clusters) with a certain probability based on their use.

#### This example illustrates how to group a set of different clinical notes based on their topics and the terminology used to describe the topics. We will model our clinical notes as a collection of unigram and bigram words. Specifically, we will represent each clinical note as a feature set of unigram and bigram frequencies found in the clinical note. We will use a matrix where each row will represent a clinical note and each column a feature (i.e. distinct unigram or bigram).

### Mixture of Multinomial Distributions

#### Text is best represented as a mixture of multinomial distributions where each topic has a particular multinomial distribution associated with it and each document in a mixture of topics. 

#### Formally, let $p(c)\space=\space\pi_c$ be the prior probability of a document containing topic c, and each topic c is represented as a multinomial distribution $p(D_i|c)$ with parameters $\mu_{jc}$, then each document becomes a mixture over topics as

#### $p(D_i)\space=\space \displaystyle\sum_{c=1}^{n_c} p(D_i|c)p(c) = \displaystyle\sum_{c=1}^{n_c} \pi_c \prod_{j=1}^{n_w}\mu_{jc}^{T_{ij}}$

### Expectation Maximization for Mixtures of Multinomials

#### The expectation maximization algorithm will allow us to fit a multinomial mixture model to our data. Our goal is to identify which documents belong to which topics and what words (unigrams and bigrams) are used to describe the topic. 

#### 1. E-Step. Compute the expectation that document $D_i$ belongs to topic (cluster) $c$

####         $\gamma_{ic} \propto \pi_c \displaystyle\prod_{j = 1}^{n_w} \mu_{jc}^{T_{ij}}$

#### $Note: \space normalize \space expectations \space over \space c$

#### where,
#### $\pi_c \equiv prior{\space}probability{\space}of{\space}document{\space}containing{\space}topic{\space}c$
#### $\mu_{jc} \equiv probability{\space}of{\space}w_j{\space}in{\space}topic{\space}c$
#### $T_{ij} \equiv count \space of \space w_j \space in \space topic \space c$


#### 2. M-Step. Update the mixture parameters. 

#### $\mu_{jc} = \frac{\sum_{i = 1}^{n_d} \gamma_{ic}T_{ij}}{\sum_{i = 1}^{n_d}\sum_{l = 1}^{n_w} \gamma_{ic}T_{il}} \equiv probability \space of \space a \space word \space being \space w_j \space in \space topic \space (cluster) \space c$ 

#### $\pi_c = \frac{1}{n_d} \displaystyle \sum_{i = 1}^{n_d} \gamma_{ic} \equiv prior \space probability \space of \space each \space cluster$

#### $Note: \space normalize \space priors \space uniformly,\space initialize \space \mu's \space to \space multinomial \space generated \space from \space uniform \space dirichlet \space distribution \space such \space that \sum_{j = 1}^{n_w} \mu_{jc} = 1$

### Implementation

#### Our implementation will consist of miipulating 4 matricies as described below.

![title](./images/MultinomialEMClustering.png)


### Environment Setup

#### First we will import some Python packages that we will use.

In [1]:
import nltk
import re
import pandas as pd
import numpy as np 
import numpy.matlib as ml
import pickle as pkl
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### Syntactic NLP Processing

#### We need a function to tokenize our text and remove noise like dates, ages, etc.

In [2]:
def tokenize(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = [ token for token in tokens if re.search('(^[a-zA-Z]+$)', token) ]
    return filtered_tokens

### Retrieving our Corpus

#### Let's pull in our corpus that we had serialized out to disk.  

In [3]:
file = open('differential-corpus.pkl','rb')
corpus = pkl.load(file)
file.close()
corpus.head()

,text,label
0,\n\n\n DATE: [**2996-12-2**] 10:25 AM\n ...,PNA
1,\n\n\n DATE: [**2850-2-14**] 10:22 PM\n ...,PNA
2,\n\n\n DATE: [**2631-10-3**] 9:52 AM\n ...,PNA
3,\n\n\n DATE: [**2584-11-21**] 11:17 AM\n ...,PNA
4,\n\n\n DATE: [**2584-11-21**] 11:17 AM\n ...,PNA


### More Syntactic Processing

#### We will want to get ride of stop words that are essentially noise. 

In [4]:
cachedStopWords = stopwords.words("english") 
noisywords = ['year', 'old', 'man', 'woman', 'ap', 'am', 'pm', 'portable', 'pa', 'lat', 'admitting', 'diagnosis', 'lateral']
cachedStopWords.extend(noisywords)
print(cachedStopWords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Generate Document-Term Frequency Counts

#### In this step we tokenize our text and remove stop words in addition to generating our frequency counts.

#### 1) how many documents are we working with and how many features (unigrams & bigrams)?

#### 2) Can you figure out what max_df and min_df is doing to our feature count?

In [5]:
corpusList = corpus['text'].tolist()
labels = []
idx = 1
for label in corpus['label'].tolist():
    labels.append(label + "-" + str(idx))
    idx = idx + 1
rank = list(range(len(labels)))

cv = CountVectorizer(lowercase=True, max_df=0.80, max_features=None, min_df=0.033,
                     ngram_range=(2, 2), preprocessor=None, stop_words=cachedStopWords,
                     strip_accents=None, tokenizer=tokenize, vocabulary=None)
SparseT = cv.fit_transform(corpusList)
print("The dimensions of our document-term matrix")
print(SparseT.shape)
print()

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['could', 'might', 'must', 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


The dimensions of our document-term matrix
(1500, 291)



### Feature Set

#### Let's take a look at our feature set.

#### 1) Do we have a lot of noise in our features?

In [6]:
lexicon = cv.get_feature_names()
print (lexicon)
print()

['abdomen contrast', 'abdomen pelvis', 'acute sob', 'adrenal glands', 'air space', 'also noted', 'amt final', 'amt underlying', 'aortic stenosis', 'approximately cm', 'atelectasis left', 'axial images', 'bilateral pleural', 'bilateral pulmonary', 'bone windows', 'cad chf', 'cardiac mediastinal', 'cardiac silhouette', 'catheter terminates', 'catheter tip', 'cc optiray', 'central line', 'central venous', 'change final', 'chest clip', 'chest compared', 'chest comparison', 'chest contrast', 'chest ct', 'chest different', 'chest history', 'chest indication', 'chest obtained', 'chest pain', 'chest physician', 'chest port', 'chest radiograph', 'chest single', 'chest w', 'chf final', 'chf reason', 'chf underlying', 'clinical indication', 'cm carina', 'compared previous', 'compared prior', 'comparison chest', 'comparison findings', 'comparison made', 'condition acute', 'condition cad', 'condition chf', 'condition copd', 'condition critical', 'condition reason', 'condition sob', 'condition yo', 

## Define EM Algorithm (E-Step & M-Step)

#### Now let's define our EM Algorithm.

#### 1) In the ExpD function why are we multiplying by $1x10^2 \space ?$  

In [7]:
  
def ExpD(T, Mu, Pi):    
    C_n = Pi.shape[1]
    D_n = T.shape[0]
    Gamma = ml.zeros((D_n, C_n))
    Gamma.astype('float64')
    for c in range(0, C_n):
        Gamma[:, c] = Pi[0][c] * ((Mu[:,c].A[:,0]*1e2)**T).prod(1)
    Gamma = Gamma / Gamma.sum(axis=1)
    return Gamma
    
def updateMu(T, Gamma):    
    C_n = Gamma.shape[1]
    W_n = T.shape[1]
    Mu = ml.zeros((W_n, C_n))
    for c in range(0, C_n):
        numerator = sum(np.multiply(Gamma[:,c],T)).T
        demoninator = sum(np.multiply(Gamma[:,c],T).sum(1))
        Mu[:,c] = numerator / demoninator
    return Mu
    
def updatePi(Gamma):    
    D_n = Gamma.shape[0]
    Pi = sum(Gamma) / D_n
    return Pi.A
    

## Let's Run it !

#### Let's start with 10 topics (clusters) and we will interate 100 times. EM converges quickly.

#### 1) Can you determine at what iteration we are starting to reach convergence?

In [8]:
T = SparseT.todense()
D_n, W_n = T.shape
C_n = 10
Pi = ml.repmat(1/C_n, 1, C_n)
Mu = ml.mat(np.random.dirichlet(np.ones(W_n), C_n).T)
for i in range(1,101):
    print('Iteration: ' + str(i)) 
    Gamma = ExpD(T, Mu, Pi)
    print(Gamma.sum(0))
    Mu = updateMu(T, Gamma)
    Pi = updatePi(Gamma)


Iteration: 1
[[ 97.62888595 149.36299945  52.08012899 280.71017234 192.98601045
  143.78559334 270.81517456 162.22984729  71.73048784  78.6706998 ]]
Iteration: 2
[[103.54998859 104.95228267  61.09883515 231.89551636 102.6989738
  347.96653652 243.41605319 174.47102067  57.36361748  72.58717556]]
Iteration: 3
[[ 91.94590855  56.36794006  62.69897842 210.77869501  75.21403833
  354.40092454 326.22605674 201.92818623  42.44982925  77.98944287]]
Iteration: 4
[[ 82.33311113  45.80561871  61.45317491 221.94337488  70.09953241
  324.4933618  371.22832211 209.31861498  33.30875213  80.01613692]]
Iteration: 5
[[ 81.82951278  43.99111783  61.13713167 209.94679858  68.8810432
  314.9589808  391.24497807 220.0018596   29.88053072  78.12804676]]
Iteration: 6
[[ 84.23553809  43.34116978  61.16849994 204.48543393  67.86455048
  309.03838571 398.29180008 225.15464366  28.51130522  77.90867311]]
Iteration: 7
[[ 85.91772855  43.6942984   61.18577802 201.5331085   67.32612853
  305.02136383 399.86059752 

Iteration: 56
[[103.70776325  43.98595327  61.43364791 162.42577258  67.14731611
  227.28663559 432.04893368 298.83111981  27.66377404  75.46908376]]
Iteration: 57
[[102.29080378  43.98608426  61.43107668 162.41154439  68.65007636
  227.22654201 431.97464123 298.90211124  27.65933801  75.46778204]]
Iteration: 58
[[102.60056774  43.98771759  61.41537196 162.39711236  68.85240144
  227.16923523 431.64462325 298.85451839  27.62716417  75.45128788]]
Iteration: 59
[[102.84145229  43.98800704  61.41384676 162.3889511   68.85931562
  227.13418307 431.43695676 298.88034627  27.60807808  75.44886302]]
Iteration: 60
[[102.95431949  43.98791216  61.41457377 162.38505494  68.86339651
  227.11211484 431.31755046 298.9171143   27.59847671  75.44948682]]
Iteration: 61
[[103.01750568  43.98781398  61.41518582 162.38309662  68.86647144
  227.09709571 431.2439131  298.94562079  27.59293787  75.450359  ]]
Iteration: 62
[[103.05713797  43.98774537  61.41562412 162.38193717  68.86870958
  227.0864881  431.

## Examination of Clusters and Terminology

#### Let's take a look at the top cluster for each clinical note and the top 20 words to distinguish this topic.

#### 1) Is there noise in the terminology? If there is how can we get ride of it?


In [9]:
clusters = Gamma.argmax(1).A
clusters = [i[0] for i in clusters]
clinicalDocuments = { 'labels': labels, 'rank': rank, 'corpus': corpusList, 'cluster': clusters }
frame = pd.DataFrame(clinicalDocuments, index = clusters , columns = ['rank', 'labels', 'corpus', 'cluster'])
grouped = frame['rank'].groupby(frame['cluster'])
topWords = Mu.T.argsort()[:, ::-1]
for i in range(C_n):
    n = len(frame.loc[i]['labels'].values.tolist())
    print('Cluster %d (%d):,' % (i+1, n), end='')
    for label in frame.loc[i]['labels'].values.tolist():
        print(' %s,' % label, end='')
    print()
    print()
    print('           Words:', end='')
    for indice in list(topWords[i, :20].A[0]):
        print(' %s (%.5f)' % (lexicon[indice], Mu[indice,i]), end=',')
    print()
    print()
    print()

Cluster 1 (103):, PNA-68, PNA-85, PNA-96, PNA-112, PNA-117, PNA-124, PNA-125, PNA-137, PNA-143, PNA-147, PNA-153, PNA-161, PNA-165, PNA-170, PNA-193, PNA-205, PNA-217, PNA-218, PNA-221, PNA-222, PNA-225, PNA-229, PNA-234, PNA-235, PNA-240, PNA-247, PNA-251, PNA-253, PNA-263, PNA-272, PNA-276, PNA-285, PNA-286, PNA-288, PNA-295, PNA-300, PNA-302, PNA-314, PNA-318, PNA-324, PNA-326, PNA-336, PNA-345, PNA-356, PNA-364, PNA-366, PNA-369, PNA-370, PNA-371, PNA-379, PNA-403, PNA-404, PNA-405, PNA-424, PNA-425, PNA-444, PNA-450, PNA-455, PNA-456, PNA-466, PNA-467, PNA-468, PNA-482, PNA-484, PNA-487, PNA-492, PNA-498, CHF-537, CHF-545, CHF-558, CHF-563, CHF-574, CHF-675, CHF-676, CHF-678, CHF-693, CHF-714, CHF-766, CHF-771, CHF-790, CHF-870, CHF-882, CHF-949, CHF-957, CHF-988, CHF-1037, CHF-1045, CHF-1058, CHF-1063, CHF-1074, CHF-1175, CHF-1176, CHF-1178, CHF-1193, CHF-1214, CHF-1266, CHF-1271, CHF-1290, CHF-1370, CHF-1382, CHF-1449, CHF-1457, CHF-1488,

           Words: lower lobe (0.07493),

Cluster 7 (439):, PNA-11, PNA-24, PNA-29, PNA-31, PNA-34, PNA-57, PNA-65, PNA-72, PNA-73, PNA-80, PNA-82, PNA-94, PNA-95, PNA-104, PNA-121, PNA-132, PNA-134, PNA-135, PNA-138, PNA-140, PNA-142, PNA-154, PNA-155, PNA-159, PNA-166, PNA-167, PNA-169, PNA-171, PNA-173, PNA-181, PNA-182, PNA-188, PNA-191, PNA-195, PNA-196, PNA-202, PNA-207, PNA-208, PNA-220, PNA-228, PNA-231, PNA-244, PNA-246, PNA-254, PNA-260, PNA-265, PNA-267, PNA-273, PNA-289, PNA-297, PNA-304, PNA-309, PNA-313, PNA-319, PNA-327, PNA-332, PNA-333, PNA-334, PNA-337, PNA-338, PNA-339, PNA-341, PNA-342, PNA-343, PNA-344, PNA-352, PNA-355, PNA-357, PNA-367, PNA-373, PNA-378, PNA-380, PNA-382, PNA-383, PNA-393, PNA-397, PNA-400, PNA-409, PNA-418, PNA-427, PNA-428, PNA-429, PNA-431, PNA-432, PNA-437, PNA-442, PNA-451, PNA-452, PNA-453, PNA-460, PNA-461, PNA-462, PNA-463, PNA-464, PNA-470, PNA-471, PNA-472, PNA-480, PNA-485, PNA-486, PNA-488, PNA-489, PNA-490, CHF-504, CHF-507, CHF-509, CHF-510, CHF-512, CHF-514, CHF-515, CHF-5

### Soft Clustering Document Examination

#### 1) Can you find clinical notes that belong to more than 1 cluster ?


In [10]:
N, C = Gamma.shape
for i in range(N):
    print('%s: ' % labels[i], end='')
    for j in range(C):
        print('C%d (%.7f): ' % (j+1, Gamma[i, j]), end='')
    print()
    print()
print()

PNA-1: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (1.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-2: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (1.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-3: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (1.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-4: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (1.0000000): 

PNA-5: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (1.0000000): 

PNA-6: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000):

PNA-86: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (1.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-87: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (1.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-88: C1 (0.0000000): C2 (0.9999193): C3 (0.0000000): C4 (0.0000000): C5 (0.0000807): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-89: C1 (0.0000000): C2 (0.9999193): C3 (0.0000000): C4 (0.0000000): C5 (0.0000807): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-90: C1 (0.0000000): C2 (1.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-91: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (1.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.000

PNA-174: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (1.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-175: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.9995339): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0004661): C9 (0.0000000): C10 (0.0000000): 

PNA-176: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (1.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-177: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0004377): C7 (0.0001751): C8 (0.9993872): C9 (0.0000000): C10 (0.0000000): 

PNA-178: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (1.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-179: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000179): C8 (0.9999821): C9 

PNA-256: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-257: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-258: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-259: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-260: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0881984): C5 (0.0000000): C6 (0.0004077): C7 (0.9036821): C8 (0.0077118): C9 (0.0000000): C10 (0.0000000): 

PNA-261: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0093487): C8 (0.9906513): C9 

PNA-336: C1 (0.9991671): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0003697): C8 (0.0004632): C9 (0.0000000): C10 (0.0000000): 

PNA-337: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0087137): C7 (0.9676786): C8 (0.0236077): C9 (0.0000000): C10 (0.0000000): 

PNA-338: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-339: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

PNA-340: C1 (0.0000000): C2 (0.0000000): C3 (0.4585043): C4 (0.0000000): C5 (0.0737991): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.4676966): 

PNA-341: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 

...

CHF-1126: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (1.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1127: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.9999972): C8 (0.0000028): C9 (0.0000000): C10 (0.0000000): 

CHF-1128: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0019352): C7 (0.9980648): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1129: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000654): C7 (0.0000000): C8 (0.0000000): C9 (0.9999346): C10 (0.0000000): 

CHF-1130: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.9999999): C7 (0.0000000): C8 (0.0000001): C9 (0.0000000): C10 (0.0000000): 

CHF-1131: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.9998388): C8 (0.0001612

CHF-1212: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0031366): C7 (0.9968634): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1213: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.9994879): C7 (0.0000119): C8 (0.0005003): C9 (0.0000000): C10 (0.0000000): 

CHF-1214: C1 (0.9999961): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000001): C8 (0.0000038): C9 (0.0000000): C10 (0.0000000): 

CHF-1215: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.9999617): C5 (0.0000000): C6 (0.0000383): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1216: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.9999117): C8 (0.0000883): C9 (0.0000000): C10 (0.0000000): 

CHF-1217: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0014952): C7 (0.9985048): C8 (0.0000000

CHF-1301: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1302: C1 (0.0000000): C2 (0.0000000): C3 (1.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1303: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (1.0000000): 

CHF-1304: C1 (0.0003848): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000555): C7 (0.0008394): C8 (0.9987203): C9 (0.0000000): C10 (0.0000000): 

CHF-1305: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1306: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000

CHF-1391: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1392: C1 (0.0000057): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.9999935): C7 (0.0000000): C8 (0.0000009): C9 (0.0000000): C10 (0.0000000): 

CHF-1393: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.9992163): C7 (0.0000000): C8 (0.0007837): C9 (0.0000000): C10 (0.0000000): 

CHF-1394: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000338): C7 (0.9999177): C8 (0.0000486): C9 (0.0000000): C10 (0.0000000): 

CHF-1395: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1396: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.9999996): C7 (0.0000000): C8 (0.0000004

CHF-1480: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.4664801): C7 (0.0000000): C8 (0.5335199): C9 (0.0000000): C10 (0.0000000): 

CHF-1481: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1482: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (1.0000000): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1483: C1 (0.0000000): C2 (0.0000000): C3 (0.3043209): C4 (0.0000000): C5 (0.0000000): C6 (0.0000000): C7 (0.1691742): C8 (0.0000000): C9 (0.0000000): C10 (0.5265049): 

CHF-1484: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.9939237): C5 (0.0000000): C6 (0.0060763): C7 (0.0000000): C8 (0.0000000): C9 (0.0000000): C10 (0.0000000): 

CHF-1485: C1 (0.0000000): C2 (0.0000000): C3 (0.0000000): C4 (0.0000000): C5 (0.0000000): C6 (0.0000023): C7 (0.9994859): C8 (0.0005118

### Soft Clustering Term Examination

#### 1) Can you find terms that belong to more than 1 cluster ?

In [11]:
W, C = Mu.shape
for i in range(W):
    print('%s: ' % lexicon[i], end='')
    for j in range(C):
        print('C%d (%.5f): ' % (j+1, Mu[i, j]), end='')
    print()
    print()
print()

abdomen contrast: C1 (0.00000): C2 (0.00000): C3 (0.01591): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.00000): C9 (0.00000): C10 (0.01346): 

abdomen pelvis: C1 (0.00000): C2 (0.00181): C3 (0.00698): C4 (0.00000): C5 (0.00000): C6 (0.00000): C7 (0.00000): C8 (0.00000): C9 (0.00000): C10 (0.01418): 

acute sob: C1 (0.00539): C2 (0.00090): C3 (0.00000): C4 (0.00000): C5 (0.00000): C6 (0.00069): C7 (0.00073): C8 (0.00841): C9 (0.00000): C10 (0.00000): 

adrenal glands: C1 (0.00000): C2 (0.00181): C3 (0.00810): C4 (0.00000): C5 (0.00482): C6 (0.00000): C7 (0.00000): C8 (0.00000): C9 (0.00382): C10 (0.00697): 

air space: C1 (0.00040): C2 (0.00000): C3 (0.00000): C4 (0.00000): C5 (0.00000): C6 (0.00091): C7 (0.00124): C8 (0.00757): C9 (0.00155): C10 (0.00312): 

also noted: C1 (0.00400): C2 (0.00000): C3 (0.00530): C4 (0.00000): C5 (0.00240): C6 (0.00000): C7 (0.00131): C8 (0.00128): C9 (0.00000): C10 (0.00096): 

amt final: C1 (0.00231): C2 (0.01221): C3 (0.01033): C4 (0

edema underlying: C1 (0.00000): C2 (0.00000): C3 (0.00000): C4 (0.01350): C5 (0.00000): C6 (0.00000): C7 (0.00120): C8 (0.00720): C9 (0.00000): C10 (0.00048): 

effusion final: C1 (0.00071): C2 (0.00000): C3 (0.00000): C4 (0.00323): C5 (0.00030): C6 (0.00223): C7 (0.00344): C8 (0.00048): C9 (0.00000): C10 (0.00000): 

effusion impression: C1 (0.00307): C2 (0.00000): C3 (0.00000): C4 (0.00197): C5 (0.00000): C6 (0.00288): C7 (0.00280): C8 (0.00034): C9 (0.00000): C10 (0.00000): 

effusion pneumothorax: C1 (0.00116): C2 (0.00000): C3 (0.00112): C4 (0.00212): C5 (0.00000): C6 (0.00126): C7 (0.00171): C8 (0.00193): C9 (0.00155): C10 (0.00000): 

effusion right: C1 (0.00501): C2 (0.00090): C3 (0.00168): C4 (0.00000): C5 (0.00180): C6 (0.00143): C7 (0.00175): C8 (0.00095): C9 (0.00620): C10 (0.00144): 

effusion underlying: C1 (0.00071): C2 (0.00000): C3 (0.00000): C4 (0.00210): C5 (0.00000): C6 (0.00036): C7 (0.00439): C8 (0.00066): C9 (0.00000): C10 (0.00000): 

effusions right: C1 (0.0015

nonionic contrast: C1 (0.00000): C2 (0.01900): C3 (0.01815): C4 (0.00000): C5 (0.00390): C6 (0.00000): C7 (0.00000): C8 (0.00000): C9 (0.01084): C10 (0.00360): 

normal limits: C1 (0.00501): C2 (0.00000): C3 (0.00000): C4 (0.00054): C5 (0.00270): C6 (0.00126): C7 (0.00292): C8 (0.00119): C9 (0.00000): C10 (0.00264): 

normal size: C1 (0.00000): C2 (0.00090): C3 (0.00168): C4 (0.00289): C5 (0.00060): C6 (0.00020): C7 (0.00327): C8 (0.00108): C9 (0.00000): C10 (0.00000): 

note made: C1 (0.00661): C2 (0.00000): C3 (0.00084): C4 (0.00000): C5 (0.00756): C6 (0.00128): C7 (0.00289): C8 (0.00095): C9 (0.00643): C10 (0.00674): 

obtained lung: C1 (0.00308): C2 (0.00000): C3 (0.00754): C4 (0.00000): C5 (0.00240): C6 (0.00000): C7 (0.00000): C8 (0.00000): C9 (0.00000): C10 (0.00505): 

opacity left: C1 (0.00549): C2 (0.00362): C3 (0.00000): C4 (0.00159): C5 (0.00150): C6 (0.00214): C7 (0.00247): C8 (0.00095): C9 (0.00000): C10 (0.00120): 

opacity right: C1 (0.00573): C2 (0.00181): C3 (0.00168)

subclavian line: C1 (0.00000): C2 (0.00000): C3 (0.00000): C4 (0.00771): C5 (0.00029): C6 (0.01342): C7 (0.00024): C8 (0.00187): C9 (0.00000): C10 (0.00000): 

supine chest: C1 (0.00157): C2 (0.00000): C3 (0.00000): C4 (0.00235): C5 (0.00000): C6 (0.00251): C7 (0.00047): C8 (0.00415): C9 (0.00000): C10 (0.00000): 

suspicious lytic: C1 (0.00000): C2 (0.00090): C3 (0.00474): C4 (0.00000): C5 (0.00540): C6 (0.00000): C7 (0.00000): C8 (0.00000): C9 (0.00310): C10 (0.00385): 

technique contiguous: C1 (0.00308): C2 (0.00271): C3 (0.00502): C4 (0.00000): C5 (0.00240): C6 (0.00000): C7 (0.00000): C8 (0.00000): C9 (0.00000): C10 (0.00265): 

technique helically: C1 (0.00039): C2 (0.00407): C3 (0.00670): C4 (0.00000): C5 (0.00212): C6 (0.00000): C7 (0.00000): C8 (0.00000): C9 (0.00769): C10 (0.00216): 

thoracic aorta: C1 (0.00421): C2 (0.00000): C3 (0.00168): C4 (0.00213): C5 (0.00300): C6 (0.00100): C7 (0.00170): C8 (0.00236): C9 (0.00000): C10 (0.00048): 

thoracic inlet: C1 (0.00000): C2 (